## Dreambooth
#### Colab implementation by David Bielejeski. Latest information on: https://github.com/JoePenna/Dreambooth-Stable-Diffusion
##### Before starting, make sure you have the appropriate Accelerator and GPU Type selected from the Runtime menu `Runtime > Change runtime type`.  A minimum of 24GB of VRAM is required so you should select the A100 GPU (40GB). Both the T4 and V100 have less than 20GB.

In [1]:
#@title Check GPU and VRAM available. (Optional)
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader
print("\n\033[92mIf the available VRAM is equal or more than 24GB, then you are good to go.\033[0m")

Tesla T4, 15360 MiB, 15101 MiB

If the available VRAM is equal or more than 24GB, then you are good to go.


In [2]:
#@title 1. Clone & Download The Repo
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion
%cd Dreambooth-Stable-Diffusion

Cloning into 'Dreambooth-Stable-Diffusion'...
remote: Enumerating objects: 1432, done.
remote: Counting objects: 100% (557/557), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 1432 (delta 350), reused 505 (delta 334), pack-reused 875 (from 1)
Receiving objects: 100% (1432/1432), 17.16 MiB | 12.52 MiB/s, done.
Resolving deltas: 100% (838/838), done.
/content/Dreambooth-Stable-Diffusion


In [ ]:
#@title 2. Build The Environment
#@markdown You might get warnings about restarting the runtime. Do this from the Runtime menu and after restarting, resume from Cell.
# !pip install numpy==1.23.1
!pip install pytorch-lightning==1.7.6
!pip install csv-logger
!pip install torchmetrics==0.11.1
!pip install torch-fidelity==0.3.0
!pip install albumentations==1.1.0
!pip install opencv-python==4.7.0.72
!pip install pudb==2019.2
!pip install omegaconf==2.1.1
!pip install pillow==9.4.0
!pip install einops==0.4.1
!pip install transformers==4.25.1
!pip install kornia==0.6.7
!pip install diffusers[training]==0.3.0
!pip install captionizer==1.0.1
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -e .
!pip install huggingface_hub
!pip install gitpython

import os
os._exit(00)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 36.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.16 requires numpy>=1.24, but you have numpy 1.23.1 which is incompatible.
albumentations 1.4.15 requires numpy>=1.24.4, but you have numpy 1.23.1 which is incompatible.
bigframes 1.22.0 requires numpy>=1.24.0, but you have numpy 1.23.1 which is incompatible.
chex 0.1.87 requires numpy>=1.24.1, but you have numpy 1.23.1 which is incompatible.
ibis-framework 9.2.0 requires numpy<3,>=1.23.2, but you have numpy 1.23.1 which is incompatible.
jax 0.4.33 requires numpy>=1.24, but you have numpy 1.23.1 which is incompatible.
jaxlib 0.4.33 requires numpy>=1.24, but you have numpy 1.23

Requested pytorch-lightning==1.7.6 from https://files.pythonhosted.org/packages/f2/22/37c64bd5b426297c71ecbb01ec2d340f013556a973a2cd6cd0aa68cda1ab/pytorch_lightning-1.7.6-py3-none-any.whl has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    torch (>=1.9.*)
           ~~~~~~^
Please use pip<24.1 if you need to use this version.
ERROR: Could not find a version that satisfies the requirement pytorch-lightning==1.7.6 (from versions: 0.0.2, 0.2, 0.2.2, 0.2.3, 0.2.4, 0.2.4.1, 0.2.5, 0.2.5.1, 0.2.5.2, 0.2.6, 0.3, 0.3.1, 0.3.2, 0.3.3, 0.3.4, 0.3.4.1, 0.3.5, 0.3.6, 0.3.6.1, 0.3.6.3, 0.3.6.4, 0.3.6.5, 0.3.6.6, 0.3.6.7, 0.3.6.8, 0.3.6.9, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.4.6, 0.4.7, 0.4.8, 0.4.9, 0.5.0, 0.5.1, 0.5.1.2, 0.5.1.3, 0.5.2, 0.5.2.1, 0.5.3, 0.5.3.1, 0.5.3.2, 0.5.3.3, 0.6.0, 0.7.1, 0.7.3, 0.7.5, 0.7.6, 0.8.1, 0.8.3, 0.8.4, 0.8.5, 0.9.0, 0.10.0, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.0.6, 1.0.7, 1.0.8, 1.1.0, 1.1.1, 1.1.2, 1.1.3, 1.1.4, 1.1.5, 1.

In [ ]:
#@title 3. Just to ensure you are in the right directory.
%cd Dreambooth-Stable-Diffusion

In [ ]:
#@title 4. Download the 1.5 SD model with the improved VAE
from IPython.display import clear_output

from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="panopstor/EveryDream",
 filename="sd_v1-5_vae.ckpt"
)

# Move the sd_v1-5_vae.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")


In [ ]:
#@title 5. Download Regularization Images
#@markdown We’ve created the following image sets
#@markdown - `man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
#@markdown - `man_unsplash` - pictures from various photographers
#@markdown - `person_ddim`
#@markdown - `woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0 <br />
#@markdown - `artstyle` - provided by Hackmans - ddim @ 50 steps, CFG 10.0 <br />

dataset="person_ddim" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "artstyle"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

# remove temp folder now it is empty.
!rm -rf Stable-Diffusion-Regularization-Images-{dataset}

clear_output()
print("✅ \033[92mRegularization Images downloaded.\033[0m")

In [ ]:
#@title 6. Training Images
#@markdown ## Upload your training images
#@markdown WARNING: Be sure to upload an even amount of images, otherwise the training inexplicably stops at 1500 steps. <br />
#@markdown - 2-3 full body
#@markdown - 3-5 upper body
#@markdown - 5-12 close-up on face  <br /> <br />
#@markdown The images should be as close as possible to the kind of images you’re trying to make (most of the time, that means no selfies).<br /><br />
#@markdown If you get an error during uploading, just manually drag your training images into the training_images folder.
from google.colab import files
from IPython.display import clear_output

# Create the directory
!rm -rf training_images
!mkdir -p training_images

# Upload the files
uploaded = files.upload()
for filename in uploaded.keys():
 updated_file_name = filename.replace(" ", "_")
 !mv "{filename}" "training_images/{updated_file_name}"
 clear_output()

# Tell the user what is going on
training_images_file_paths = !find training_images/*
if len(training_images_file_paths) == 0:
 print("❌ \033[91mno training images found. Please upload images to training_images.\033[0m")
else:
 print("✅ \033[92m" + str(len(training_images_file_paths)) + " training images found.\033[0m")



In [ ]:
#@title 7. Final Setup & Training

#@markdown This isn't used for training, just used to name the folders etc
project_name = "project_name" #@param {type:"string"}

#@markdown This is the unique token i.e. you can use a nonsensical word like zwx or your name.
token = "firstNamelastName" #@param {type:"string"}

#@markdown Match class_word to the category of the regularization images you chose above.
class_word = "person" #@param ["man", "person", "woman"] {allow-input: true}

# MAX STEPS
#@markdown How many steps do you want to train for?
max_training_steps = 2000 #@param {type:"integer"}

#@markdown If you are training a person's face, set this to True
i_am_training_a_persons_face = True #@param {type:"boolean"}
flip_p_arg = 0.0 if i_am_training_a_persons_face else 0.5

#@markdown Would you like to save a model every X steps? (Example: 250 would output a trained model at 250, 500, 750 steps, etc)
save_every_x_steps = 0 #@param {type:"integer"}


reg_data_root = "/content/Dreambooth-Stable-Diffusion/regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --project_name "{project_name}" \
 --debug False \
 --max_training_steps {max_training_steps} \
 --token "{token}" \
 --training_model "model.ckpt" \
 --training_images "/content/Dreambooth-Stable-Diffusion/training_images" \
 --regularization_images "{reg_data_root}" \
 --class_word "{class_word}" \
 --flip_p {flip_p_arg} \
 --save_every_x_steps {save_every_x_steps}

In [ ]:
#@title 8. Save model into google drive
#@markdown This is often much faster than a manual download.  it will also save you compute units. <br />
from google.colab import drive
drive.mount('/content/drive')

# copy all ckpt files to google drive root dir
!cp trained_models/*.ckpt /content/drive/MyDrive